In [ ]:
import pandas as pd
from pymarc import Field, parse_xml_to_array
import verovio

In [2]:
verovio_tk = verovio.toolkit()
verovio_tk.setOptions({
    "adjustPageHeight": True,
    "adjustPageWidth": True,
    "header": "none",
    "footer": "none",
    "pageMarginBottom" :0,
    "pageMarginLeft" :0,
    "pageMarginRight" :0,
    "pageMarginTop" :0,

})

True

In [3]:
records = parse_xml_to_array("../data/rism_entries.xml")

In [4]:
def get_incipit(r):
    incipits = r.get_fields("031")
    if not incipits:
        return ""
    res = []
    for ind, incipit in enumerate(incipits):
        label = "{a}.{b}.{c} {m}, {r}".format(
            a=incipit.get("a", ""),  # number of work
            b=incipit.get("b", ""),  # number of movement
            c=incipit.get("c", ""),  # number of excerpt
            m=incipit.get("m", ""),  # instrument
            r=incipit.get("r", "")  # key
        )
        text = incipit.get("t", "")   # text
        pae = "@clef:{g}\n@keysig:{n}\n@timesig:{o}\n@data:{p}".format(
            g=incipit.get("g", ""),  # clef
            n=incipit.get("n", ""),  # key signature
            o=incipit.get("o", ""),  # time signature
            p=incipit.get("p", "")   # notes
        )
        res.append((ind, label, text, pae))

    return res

In [5]:
record_list = [(
    r["001"].data,  # ID
    r["240"]["a"],  # title
    r.get("730", Field(tag="730")).get("a", ""),  # other title
    r.get("650", Field(tag="650")).get("a", ""),  # genre
    r["852"]["a"],  # library
    r["852"]["c"],  # shelfmark
    get_incipit(r)
) for r in records]

df_works = (
    pd.DataFrame.from_records(
    record_list,
    columns=["rism_id", "title", "title_alt", "genre", "siglum", "shelfmark", "incipit_data"]
    )
    .sort_values(["siglum", "shelfmark"])
    .reset_index(drop=True)
)

In [6]:
incipit_list = []
for work_id, incipits in df_works.incipit_data.items():
    for inc_id, label, text, data in incipits:
        verovio_tk.loadData(data)
        outfile = f"../data_generated/rism_incipits/{work_id}-{inc_id}.svg"
        success = verovio_tk.renderToSVGFile(outfile)
        if not success:
            print(work_id, inc_id, "failed", sep=" ")
        incipit_list.append((work_id, inc_id, label, text, outfile))

[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PAE: A clef is required. (clef input key)
[Warning] PA

In [7]:
df_inc = pd.DataFrame.from_records(
    incipit_list,
    columns=["work_id", "incipit_id", "label", "text", "file"]
)

In [8]:
df_works.to_csv("../data_generated/rism_entries.csv", index_label="work_id")
df_inc.to_csv("../data_generated/rism_incipits.csv", index=False)